In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

I0202 235046.167 _utils_internal.py:234] NCCL_DEBUG env var is set to None


I0202 235046.168 _utils_internal.py:252] NCCL_DEBUG is forced to WARN from None


I0202 235048.446 font_manager.py:1349] generated new fontManager


In [2]:
# load data

words = open("/home/yucongguo/names.txt", "r").read().splitlines()
print(words[:8])
print(f"length of words = {len(words)}")

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
length of words = 32033


In [6]:
# build the vocabulary of characters and mapping to/from integers

char = sorted(list(set(''.join(words))))
str_to_int = {string: index+1 for index, string in enumerate(char)}
str_to_int['.'] = 0
int_to_str = {index+1: string for index, string in enumerate(char)}
int_to_str[0] = '.'
vocab_size = len(int_to_str)
print(f"str_to_int : {str_to_int}\n")
print(f"int_to_str : {int_to_str}\n")
print(f"vocab_size : {vocab_size}")

str_to_int : {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}

int_to_str : {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}

vocab_size : 27


In [9]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    #input: list of wards
    #output: X,Y

    x = []
    y = []
    for w in words:
        context = [0] * block_size #assume all word start with "..."
        for character in w+'.':
            next_char_index = str_to_int[character]
            x.append(context)
            y.append(next_char_index)
            context = context[1:] + [next_char_index]

    x = torch.tensor(x)
    y = torch.tensor(y)

    return x, y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

x_train,y_train = build_dataset(words[:n1])
x_val,y_val = build_dataset(words[n1:n2])
x_test,y_test = build_dataset(words[n2:])

print(f"x_train shape : {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape : {x_val.shape}, y_val shape: {y_val.shape}")
print(f"x_test shape : {x_test.shape}, y_test shape: {y_test.shape}")

x_train shape : torch.Size([182437, 3]), y_train shape: torch.Size([182437])
x_val shape : torch.Size([22781, 3]), y_val shape: torch.Size([22781])
x_test shape : torch.Size([22928, 3]), y_test shape: torch.Size([22928])


In [79]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item() #return true of false
  app = torch.allclose(dt, t.grad) #return true of false
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

## forward pass

In [92]:
# define parameters

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP
#vocal_size = 27
#block_size = 3

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [103]:
# forward pass

batch_size = 32
ix = torch.randint(0, x_train.shape[0],(batch_size,), generator = g)
x_train_batch, y_train_batch = x_train[ix], y_train[ix] # batch X,Y

emb = C[x_train_batch] # embed the characters into vectors
embcat = emb.view(emb.shape[0],-1) # reshape into a tensor of size (32,30)
# Linear layer 1
hidden_pre_batchnorm = embcat @ W1 + b1 # hidden layer pre-BatchNorm  (64,1) = (32,30) *(30,64) + (64,1)
# BatchNorm layer
bnmeani = 1/batch_size * hidden_pre_batchnorm.sum(0, keepdim=True) #minibatch mean
bndiff = hidden_pre_batchnorm - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hidden_pre_act = bngain * bnraw + bnbias # scale and shift normalized layer

# Non-linearity
h = torch.tanh(hidden_pre_act) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
#cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
#probs  = norm_logits.exp() / norm_logits.exp().sum(1, keepdims=True) #softmax
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log() #(32,27)
loss = -logprobs[range(batch_size), y_train_batch].mean() # (x1+x2+x3+x4+..+x32) / 32


print(f"loss = {loss}")

loss = 3.4124536514282227


## backprop 

In [105]:
# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hidden_pre_act, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hidden_pre_batchnorm, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss


tensor(3.4125, grad_fn=<NegBackward0>)

In [106]:
#backprop of softmax function


loss = -logprobs[range(batch_size), y_train_batch].mean() # (x1+x2+x3+x4+..+x32) / 32

d_loss= 1

'''
loss = -logprobs[range(batch_size), y_train_batch].mean() # (x1+x2+x3+x4+..+x32) / 32
'''
d_logprobs = torch.zeros_like(logprobs)
d_logprobs[range(batch_size), y_train_batch] = -1.0 / batch_size #since (x1+x2+x3+x4+..+x32) / 32, then dx1 = 1/32
d_logprobs = d_loss *d_logprobs #(32,27)

'''
logprobs = probs.log() 
logprobs: (32,27)
probs: (32,27)
'''
d_probs = d_logprobs *  (1/probs)   # (32,27) = (32,27) * 1/(32,27) | y=logx,  dy/dx = 1/x

'''
probs = counts * counts_sum_inv  
probs: (32,27)
counts: (32,27)
counts_sum_inv: (32,1)
'''
d_counts = d_probs * counts_sum_inv  # (32,27) * (32,1) = (32,27 )
d_counts_sum_inv = (d_probs * counts).sum(1, keepdim = True) #(32,1)

'''
counts_sum_inv = counts_sum**-1 
counts_sum_inv: (32,1)
counts_sum: (32,1)
'''
d_counts_sum = d_counts_sum_inv * (-counts_sum**-2)  #(32,1)

'''
counts_sum = counts.sum(1, keepdims=True)
counts_sum: (32,1)
counts: (32,27)
'''
d_counts += d_counts_sum * torch.ones_like(counts)  #(32,1) * (32,27) = (32,27)
# a11 a12 a13 ----> b1 = a11+a12+a13.     therefore, the deravete is 1
# a21 a22 a23 ----> b2 = a21+a22+a23
# a31 a32 a33 ----> b3 = a31+a32+a33

'''
counts = norm_logits.exp()
counts: (32,27)
norm_logits : (32,27)
'''
d_norm_logits = d_counts * counts

'''
norm_logits = logits - logit_maxes # subtract max for numerical stability
(32,27)
'''
d_logits = d_norm_logits.clone()

cmp('logprobs', d_logprobs, logprobs)
cmp('probs', d_probs, probs)
cmp('counts_sum_inv', d_counts_sum_inv, counts_sum_inv)
cmp('counts_sum', d_counts_sum, counts_sum)
cmp('counts', d_counts, counts)
cmp('norm_logits', d_norm_logits, norm_logits)
cmp('logits', d_logits, logits)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: False | approximate: True  | maxdiff: 7.2177499532699585e-09


In [118]:
# backprop of model


'''
logits = h @ W2 + b2 
logits: (32,27)
h : (32,64)
w2:(62,27)
b2: (32,1)
'''
d_h = d_logits @ W2.T #(32,27) @ (27,64) = (32,64)
d_W2 = h.T @ d_logits #(64,32) @ (32,27) = (64,27)
d_b2 = d_logits.sum(0)

'''
h = torch.tanh(hidden_pre_act) # hidden layer
h : (32,64)
hidden_pre_act : (32,64)
'''
d_hidden_pre_act = (1.0 - h**2) * d_h

'''
hidden_pre_act = bngain * bnraw + bnbias # scale and shift normalized layer
hidden_pre_act : (32,64)
bngain : (1,64)
bnraw : (32,64)
bnbias : (1,64)
'''
d_bngain = (bnraw * d_hidden_pre_act).sum(0, keepdim=True) 
d_bnraw = bngain * d_hidden_pre_act
d_bnbias = d_hidden_pre_act.sum(0, keepdim=True)

'''
bnraw = bndiff * bnvar_inv
bnraw : (32,64)
bndiff : (32,64)
bnvar_inv : (1,64)
'''
d_bndiff = bnvar_inv * d_bnraw
d_bnvar_inv = (bndiff * d_bnraw).sum(0, keepdim=True)

'''
bnvar_inv = (bnvar + 1e-5)**-0.5
bnvar_inv : (1,64)
bnvar : (1,64)
'''
d_bnvar = (-0.5*(bnvar + 1e-5)**-1.5) * d_bnvar_inv

'''
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar : (1,64)
bndiff2: (32,64)
'''
d_bndiff2 = (1.0/(batch_size-1))*torch.ones_like(bndiff2) * d_bnvar

'''
bndiff2 = bndiff**2
'''
d_bndiff += (2*bndiff) * d_bndiff2

'''
bndiff = hidden_pre_batchnorm - bnmeani
bndiff: (32,64)
hidden_pre_batchnorm: (32,64)
bnmeani: (1,64)
'''
d_hidden_pre_batchnorm = d_bndiff.clone()
d_bnmeani = (-d_bndiff).sum(0)

'''
bnmeani = 1/batch_size * hidden_pre_batchnorm.sum(0, keepdim=True) #minibatch mean
bnmeani: (1,64)
hidden_pre_batchnorm: (32,64)
'''
d_hidden_pre_batchnorm += 1.0/batch_size * (torch.ones_like(hidden_pre_batchnorm) * d_bnmeani)

'''
hidden_pre_batchnorm = embcat @ W1 + b1 # hidden layer pre-BatchNorm  (64,1) = (32,30) *(30,64) + (64,1)
'''
d_embcat = d_hidden_pre_batchnorm @ W1.T
d_W1 = embcat.T @ d_hidden_pre_batchnorm
d_b1 = d_hidden_pre_batchnorm.sum(0)

'''
embcat = emb.view(emb.shape[0],-1) # reshape into a tensor of size (32,30)
'''
d_emb = d_embcat.view(emb.shape)

'''
emb = C[x_train_batch] # embed the characters into vectors
'''
d_C = torch.zeros_like(C)
for k in range(x_train_batch.shape[0]):
    for j in range(x_train_batch.shape[1]):
        ix = x_train_batch[k,j]
        d_C[ix] += d_emb[k,j]

cmp('h', d_h, h)
cmp('W2', d_W2, W2)
cmp('b2', d_b2, b2)
cmp('hpreact', d_hidden_pre_act, hidden_pre_act)
cmp('bngain', d_bngain, bngain)
cmp('bnbias', d_bnbias, bnbias)
cmp('bnraw', d_bnraw, bnraw)
cmp('bnvar_inv', d_bnvar_inv, bnvar_inv)
cmp('bnvar', d_bnvar, bnvar)
cmp('bndiff2', d_bndiff2, bndiff2)
cmp('bndiff', d_bndiff, bndiff)
cmp('bnmeani', d_bnmeani, bnmeani)
cmp('hprebn', d_hidden_pre_batchnorm, hidden_pre_batchnorm)
cmp('embcat', d_embcat, embcat)
cmp('W1', d_W1, W1)
cmp('b1', d_b1, b1)
cmp('emb', d_emb, emb)
cmp('C', d_C, C)

h               | exact: False | approximate: True  | maxdiff: 1.6298145055770874e-09
W2              | exact: False | approximate: True  | maxdiff: 1.30385160446167e-08
b2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
hpreact         | exact: False | approximate: True  | maxdiff: 1.57160684466362e-09
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnbias          | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09
bnraw           | exact: False | approximate: True  | maxdiff: 1.6298145055770874e-09
bnvar_inv       | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
bnvar           | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-09
bndiff2         | exact: False | approximate: True  | maxdiff: 3.637978807091713e-11
bndiff          | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-09
bnmeani         | exact: False | approximate: True  | maxdiff

## train model with manual backprop

In [121]:
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 50000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, x_train.shape[0], (batch_size,), generator=g)
    Xb, Yb = x_train[ix], y_train[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

  #   if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #     break

12297
      0/  50000: 3.7521


  10000/  50000: 1.8142


  20000/  50000: 1.8368


  30000/  50000: 2.4078


  40000/  50000: 2.4239


In [123]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(int_to_str[i] for i in out))

carlaviano.
havith.
mili.
taty.
salani.
emmahnelle.
diah.
pareei.
nellairickaiiv.
kaleigh.
ham.
poin.
quinn.
sulio.
alianni.
watell.
dearixi.
jace.
pirra.
mel.
